## Welcome to your notebook.


Conexión API Libelium

In [1]:
import requests as req
from datetime import datetime, timedelta
import pandas as pd  # ✅ Agregado para manejar DataFrame

#hoy = datetime.utcnow().date()
#D_INIT = hoy.strftime('%Y-%m-%d')
#D_END = (hoy + timedelta(days=1)).strftime('%Y-%m-%d')

# --- Parámetros customizables ---
USERNAME = "masantizof@gmail.com"
PASSWORD = "Fuentes.*2025"

#DATE_INIT = "2025-09-19"
#DATE_END = "2025-09-20"

# Obtener la fecha actual en UTC
hoy = datetime.utcnow().date()
# Calcular la fecha del día anterior
ayer = hoy - timedelta(days=1)
# Formatear las fechas de inicio y fin para el filtro
D_INIT = ayer.strftime('%Y-%m-%d')
D_END = hoy.strftime('%Y-%m-%d')
DATE_INIT = D_INIT
DATE_END = D_END
NUM_RECORDS_PER_MEASURE = 1

# --- Variables de Fiware ---
TENANT = "sistemas_inteligentes_de_transporte_deviteck_sas"
SCOPE = "/"

# --- IRIS API ---
IRIS_API = "https://api.iris360iot.com"
IRIS_ENDPOINTS = {
    "login": "/api/public/v1/login",
    "get_timeseries": "/api/public/v1/timeseries",
    "paginate_entities": "/api/public/v1/entities/paginate",
    "last_data": "/api/public/v1/last-data/entities/{urn}",
}

# --- Payloads ---
PAGINATION_PAYLOAD = {
    "orderBy": "entities.urn",
    "orderDirection": False,
    "page": 1,
    "paginationSize": 1,
}

TIMESERIES_PAYLOAD = {
    "device_ids": [],
    "measure_ids": None,
    "options": {
        "order": "desc",
        "end_date": (
            datetime.strptime(DATE_END, "%Y-%m-%d").isoformat() + "Z"
            if DATE_END
            else datetime.now().isoformat() + "Z"
        ),
        "start_date": datetime.strptime(DATE_INIT, "%Y-%m-%d").isoformat() + "Z",
        "limit": NUM_RECORDS_PER_MEASURE,
        "tenant": TENANT,
        "scope": SCOPE,
    },
}

# --- Funciones API IRIS ---
def get_token():
    response = req.post(
        IRIS_API + IRIS_ENDPOINTS["login"],
        data={"username": USERNAME, "password": PASSWORD},
    )
    response.raise_for_status()
    return response.json().get("token")

def get_organization_total_number_of_entities(token):
    response = req.post(
        IRIS_API + IRIS_ENDPOINTS["paginate_entities"],
        headers={"Authorization": f"Bearer {token}"},
        json=PAGINATION_PAYLOAD,
    )
    response.raise_for_status()
    return response.json().get("count", 100000)

def get_organization_urn_list(token):
    max = get_organization_total_number_of_entities(token)
    PAGINATION_PAYLOAD["paginationSize"] = max

    response = req.post(
        IRIS_API + IRIS_ENDPOINTS["paginate_entities"],
        headers={"Authorization": f"Bearer {token}"},
        json=PAGINATION_PAYLOAD,
    )
    response.raise_for_status()

    def get_urn(entity):
        return entity.get("urn")

    return [get_urn(entity) for entity in response.json().get("rows", [])]

def get_timeseries(token, payload):
    response = req.post(
        IRIS_API + IRIS_ENDPOINTS["get_timeseries"],
        headers={"Authorization": f"Bearer {token}"},
        json=payload,
    )
    response.raise_for_status()
    return response.json()

def parse_timeseries(timeserie):
    parsed_data = {}
    urn = ""
    for data in timeserie["time_series"]:
        urn = data["device_id"]
        parsed_data[data["measure_id"]] = data["values"][0]["value"]
    return urn, parsed_data if parsed_data else None

# --- Ejecución principal ---
if __name__ == "__main__":
    token = get_token()
    urns = get_organization_urn_list(token)

    payload = []
    for urn in urns:
        timeseries_payload = TIMESERIES_PAYLOAD.copy()
        timeseries_payload["device_ids"] = [urn]
        payload.append(timeseries_payload)

    timeseries = get_timeseries(token, payload)

    parsed_timeseries = {}
    for timeserie in timeseries:
        id, data = parse_timeseries(timeserie)
        if id and data:
            parsed_timeseries[id] = data

    print(parsed_timeseries)

    # ✅ Guardar en un DataFrame
    data_list = []
    for device_id, medidas in parsed_timeseries.items():
        row = {"device_id": device_id}
        row.update(medidas)
        data_list.append(row)

    Datos_Lib = pd.DataFrame(data_list)
    #print("\n✅ DataFrame creado:\n")
    #print(Datos_Lib)


{'urn:ngsi-ld:Device:Device_670': {'name': 'Device HOP9454c54a1c86', 'cellularApnUser': '', 'cellularEnabled': 'true', 'commands': [], 'firmwareVersion': 'v1.1.4', 'modelName': 'SPT-CM-4G BAT', 'wifiEnabled': 'false', 'cellularApn': 'libelium.dt.iot', 'serialNumber': '9454c54a1c86', 'status': 'connected', 'updateFirmwareDownloadProgress': 0, 'updateFirmwareResult': 0, 'updateFirmwareState': 0, 'location': {'type': 'Point', 'coordinates': [-74.656631, 5.464782]}}, 'urn:ngsi-ld:Device:Device_671': {'name': 'Device HOPc05d89a0fb1a', 'cellularApnUser': 'null', 'cellularEnabled': 'false', 'commands': [], 'firmwareVersion': 'v1.1.4', 'modelName': 'SPT-AQ-4G BAT-AWD-PM-SPL2', 'wifiEnabled': 'false', 'cellularApn': 'null', 'serialNumber': 'c05d89a0fb1a', 'status': 'connected', 'updateFirmwareDownloadProgress': 0, 'updateFirmwareResult': 0, 'updateFirmwareState': 0, 'location': {'type': 'Point', 'coordinates': [-74.629509, 5.756509]}}, 'urn:ngsi-ld:Device:Device_672': {'name': 'Device HOPc05d89

In [2]:
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import pandas as pd


# Lista de IDs por categoría
tierra_ids = ['urn:ngsi-ld:TEROS12:ONE2038353453325019003F0023:ONS003F002E3234383113473231', 'urn:ngsi-ld:TEROS12:ONE2038353453325019003E0021:ONS001900253234383114473231']
personas_ids = ['urn:ngsi-ld:CrowdFlowObserved:HOP9454c54a1c86_CFO']
tiempo_ids = [
    'urn:ngsi-ld:WeatherObserved:HOPc05d89a1332a_WTO', 'urn:ngsi-ld:WeatherObserved:HOPc05d89a0fb1a_WTO', 'urn:ngsi-ld:WeatherObserved:HOPc05d89a13c16_WTO',
    'urn:ngsi-ld:WeatherObserved:HOP9454c54a124a_WTO', 'urn:ngsi-ld:WeatherObserved:HOPc05d89a0f736_WTO'
]
ruido_ids = [
    'urn:ngsi-ld:NoiseLevelObserved:HOPc05d89a1332a_NLO', 'urn:ngsi-ld:NoiseLevelObserved:HOPc05d89a0fb1a_NLO', 'urn:ngsi-ld:NoiseLevelObserved:HOPc05d89a13c16_NLO',
    'urn:ngsi-ld:NoiseLevelObserved:HOP9454c54a124a_NLO', 'urn:ngsi-ld:NoiseLevelObserved:HOPc05d89a0f736_NLO'
]
aire_ids = [
    'urn:ngsi-ld:AirQualityObserved:HOPc05d89a1332a_AQO', 'urn:ngsi-ld:AirQualityObserved:HOPc05d89a0fb1a_AQO', 'urn:ngsi-ld:AirQualityObserved:HOPc05d89a13c16_AQO',
    'urn:ngsi-ld:AirQualityObserved:HOP9454c54a124a_AQO', 'urn:ngsi-ld:AirQualityObserved:HOPc05d89a0f736_AQO'
]

# Crear los DataFrames
df_tierra = Datos_Lib[Datos_Lib['device_id'].isin(tierra_ids)].copy()
df_personas = Datos_Lib[Datos_Lib['device_id'].isin(personas_ids)].copy()
df_tiempo = Datos_Lib[Datos_Lib['device_id'].isin(tiempo_ids)].copy()
df_ruido = Datos_Lib[Datos_Lib['device_id'].isin(ruido_ids)].copy()
df_aire = Datos_Lib[Datos_Lib['device_id'].isin(aire_ids)].copy()


DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 51.7 MB/s eta 0:00:00:00:01


In [3]:
# Definir campos de dataframe y añadir FechaMedicion
FechaMedicion = datetime.now()

# df_tierra
df_tierra = df_tierra[['device_id','r_loop','location','r_6_cond','r_6_rwc','r_6_tmp','r_ts','latitudeLocation','longitudeLocation']]
df_tierra['longitudeLocation'] = df_tierra['location'].apply(lambda x: x['coordinates'][0])
df_tierra['latitudeLocation'] = df_tierra['location'].apply(lambda x: x['coordinates'][1])
df_tierra['FechaMedicion'] = FechaMedicion
df_tierra = df_tierra.drop(columns=['location'])
df_tierra['r_ts'] = pd.to_datetime(df_tierra['r_ts'], unit='s')

# df_personas
df_personas = df_personas[['peopleCountMediumInterval','peopleCountLongInterval','peopleCountShortInterval','location','latitudeLocation','longitudeLocation']]
df_personas['longitudeLocation'] = df_personas['location'].apply(lambda x: x['coordinates'][0])
df_personas['latitudeLocation'] = df_personas['location'].apply(lambda x: x['coordinates'][1])
df_personas['FechaMedicion'] = FechaMedicion
df_personas = df_personas.drop(columns=['location'])

# df_tiempo
df_tiempo = df_tiempo[['relativeHumidity','temperature','atmosphericPressure','location','latitudeLocation','longitudeLocation']]
df_tiempo['longitudeLocation'] = df_tiempo['location'].apply(lambda x: x['coordinates'][0])
df_tiempo['latitudeLocation'] = df_tiempo['location'].apply(lambda x: x['coordinates'][1])
df_tiempo['FechaMedicion'] = FechaMedicion
df_tiempo = df_tiempo.drop(columns=['location'])

# df_ruido
df_ruido = df_ruido[['LA99','LA1','LAeq','LAmax','LA90','LA50','LAmin','LA10','location','latitudeLocation','longitudeLocation']]
df_ruido['longitudeLocation'] = df_ruido['location'].apply(lambda x: x['coordinates'][0])
df_ruido['latitudeLocation'] = df_ruido['location'].apply(lambda x: x['coordinates'][1])
df_ruido['FechaMedicion'] = FechaMedicion
df_ruido = df_ruido.drop(columns=['location'])

# df_aire
df_aire = df_aire[['co','no2','o3','tsp','pm10','pm4','pm1','tpc','pm25','so2','location','latitudeLocation','longitudeLocation']]
df_aire['longitudeLocation'] = df_aire['location'].apply(lambda x: x['coordinates'][0])
df_aire['latitudeLocation'] = df_aire['location'].apply(lambda x: x['coordinates'][1])
df_aire['FechaMedicion'] = FechaMedicion
df_aire = df_aire.drop(columns=['location'])

# Función para convertir a GeoDataFrame
def convertir_a_geodataframe(df, lat_col='latitudeLocation', lon_col='longitudeLocation'):
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf

# Convertir a GeoDataFrames
df_tierra = convertir_a_geodataframe(df_tierra)
df_personas = convertir_a_geodataframe(df_personas)
df_tiempo = convertir_a_geodataframe(df_tiempo)
df_ruido = convertir_a_geodataframe(df_ruido)
df_aire = convertir_a_geodataframe(df_aire)

In [4]:
df_aire

,co,no2,o3,tsp,pm10,pm4,pm1,tpc,pm25,so2,latitudeLocation,longitudeLocation,FechaMedicion,geometry
11,800.114075,65.193497,53.783001,16.186625,13.293651,6.440477,1.429415,22.286915,3.008117,15.3838,5.464729,-74.656570,2025-09-23 02:57:11.963211,POINT (-74.65657 5.46473)
15,322.136505,58.306599,50.936699,20.206617,11.359431,5.615407,1.971101,27.375463,3.746722,16.6609,5.756442,-74.629547,2025-09-23 02:57:11.963211,POINT (-74.62955 5.75644)
19,284.035095,41.187801,61.486198,26.992527,6.395680,2.850642,1.435942,24.101934,2.088166,2.8410,5.499393,-74.603424,2025-09-23 02:57:11.963211,POINT (-74.60342 5.49939)
23,128.223602,40.284901,40.173000,9.965553,4.530719,2.022619,1.228515,20.940685,1.601661,6.9571,5.470923,-74.595497,2025-09-23 02:57:11.963211,POINT (-74.5955 5.47092)
27,282.338593,43.718300,41.873901,0.104580,0.102332,0.102294,0.076531,1.098338,0.101281,5.0428,5.737881,-74.561363,2025-09-23 02:57:11.963211,POINT (-74.56136 5.73788)


In [5]:
from arcgis import GIS
from arcgis.features import FeatureLayerCollection, FeatureSet
import pandas as pd
import geopandas as gpd
from datetime import datetime
import logging
from shapely.geometry import Point
import numpy as np
import time

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Autenticación en ArcGIS Online
try:
    gis = GIS("https://www.arcgis.com", "aplicaciones_puertosalgar", "aplicaciones$2025*01")
    me = gis.users.me
    logger.info(f"Autenticado como: {me.username}")
except Exception as e:
    logger.error(f"Error de autenticación: {str(e)}")
    raise

# Definir los nombres de los servicios
SERVICE_NAMES = {
    'tierra': "TierraSensorData",
    'personas': "PersonasSensorData",
    'tiempo': "TiempoSensorData",
    'ruido': "RuidoSensorData",
    'aire': "AireSensorData"
}

# 1. Definir el sufijo para vistas (FALTA PREVIA)
VIEW_SUFFIX = "_View"

# 2. Definir esquemas detallados para cada capa
SCHEMAS = {
    'tierra': {
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID"},
            {"name": "device_id", "type": "esriFieldTypeString", "alias": "Device ID", "length": 100},
            {"name": "r_loop", "type": "esriFieldTypeDouble", "alias": "Loop Resistance"},
            {"name": "r_6_cond", "type": "esriFieldTypeDouble", "alias": "Conductividad"},
            {"name": "r_6_rwc", "type": "esriFieldTypeDouble", "alias": "Water Content"},
            {"name": "r_6_tmp", "type": "esriFieldTypeDouble", "alias": "Temperature"},
            {"name": "r_ts", "type": "esriFieldTypeDate", "alias": "Timestamp"},
            {"name": "FechaMedicion", "type": "esriFieldTypeDate", "alias": "Fecha de Medición"},
            {"name": "latitudeLocation", "type": "esriFieldTypeDouble", "alias": "Latitude"},
            {"name": "longitudeLocation", "type": "esriFieldTypeDouble", "alias": "Longitude"}
        ],
        "geometryType": "esriGeometryPoint",
        "spatialReference": {"wkid": 4326},
        "name": "TierraSensor_Layer",
        "description": "Datos de sensores de tierra"
    },
    'personas': {
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID"},
            {"name": "peopleCountMediumInterval", "type": "esriFieldTypeInteger", "alias": "Personas (Medio)"},
            {"name": "peopleCountLongInterval", "type": "esriFieldTypeInteger", "alias": "Personas (Largo)"},
            {"name": "peopleCountShortInterval", "type": "esriFieldTypeInteger", "alias": "Personas (Corto)"},
            {"name": "FechaMedicion", "type": "esriFieldTypeDate", "alias": "Fecha de Medición"},
            {"name": "latitudeLocation", "type": "esriFieldTypeDouble", "alias": "Latitude"},
            {"name": "longitudeLocation", "type": "esriFieldTypeDouble", "alias": "Longitude"}
        ],
        "geometryType": "esriGeometryPoint",
        "spatialReference": {"wkid": 4326},
        "name": "PersonasSensor_Layer",
        "description": "Datos de conteo de personas"
    },
    'tiempo': {
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID"},
            {"name": "relativeHumidity", "type": "esriFieldTypeDouble", "alias": "Humedad Relativa"},
            {"name": "temperature", "type": "esriFieldTypeDouble", "alias": "Temperatura"},
            {"name": "atmosphericPressure", "type": "esriFieldTypeDouble", "alias": "Presión Atmosférica"},
            {"name": "FechaMedicion", "type": "esriFieldTypeDate", "alias": "Fecha de Medición"},
            {"name": "latitudeLocation", "type": "esriFieldTypeDouble", "alias": "Latitude"},
            {"name": "longitudeLocation", "type": "esriFieldTypeDouble", "alias": "Longitude"}
        ],
        "geometryType": "esriGeometryPoint",
        "spatialReference": {"wkid": 4326},
        "name": "TiempoSensor_Layer",
        "description": "Datos meteorológicos"
    },
    'ruido': {
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID"},
            {"name": "LA99", "type": "esriFieldTypeDouble", "alias": "LA99"},
            {"name": "LA1", "type": "esriFieldTypeDouble", "alias": "LA1"},
            {"name": "LAeq", "type": "esriFieldTypeDouble", "alias": "LAeq"},
            {"name": "LAmax", "type": "esriFieldTypeDouble", "alias": "LAmax"},
            {"name": "LA90", "type": "esriFieldTypeDouble", "alias": "LA90"},
            {"name": "LA50", "type": "esriFieldTypeDouble", "alias": "LA50"},
            {"name": "LAmin", "type": "esriFieldTypeDouble", "alias": "LAmin"},
            {"name": "LA10", "type": "esriFieldTypeDouble", "alias": "LA10"},
            {"name": "FechaMedicion", "type": "esriFieldTypeDate", "alias": "Fecha de Medición"},
            {"name": "latitudeLocation", "type": "esriFieldTypeDouble", "alias": "Latitude"},
            {"name": "longitudeLocation", "type": "esriFieldTypeDouble", "alias": "Longitude"}
        ],
        "geometryType": "esriGeometryPoint",
        "spatialReference": {"wkid": 4326},
        "name": "RuidoSensor_Layer",
        "description": "Datos de niveles de ruido"
    },
    'aire': {
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID"},
            {"name": "co", "type": "esriFieldTypeDouble", "alias": "CO"},
            {"name": "no2", "type": "esriFieldTypeDouble", "alias": "NO2"},
            {"name": "o3", "type": "esriFieldTypeDouble", "alias": "O3"},
            {"name": "tsp", "type": "esriFieldTypeDouble", "alias": "TSP"},
            {"name": "pm10", "type": "esriFieldTypeDouble", "alias": "PM10"},
            {"name": "pm4", "type": "esriFieldTypeDouble", "alias": "PM4"},
            {"name": "pm1", "type": "esriFieldTypeDouble", "alias": "PM1"},
            {"name": "tpc", "type": "esriFieldTypeDouble", "alias": "TPC"},
            {"name": "pm25", "type": "esriFieldTypeDouble", "alias": "PM2.5"},
            {"name": "so2", "type": "esriFieldTypeDouble", "alias": "SO2"},
            {"name": "FechaMedicion", "type": "esriFieldTypeDate", "alias": "Fecha de Medición"},
            {"name": "latitudeLocation", "type": "esriFieldTypeDouble", "alias": "Latitude"},
            {"name": "longitudeLocation", "type": "esriFieldTypeDouble", "alias": "Longitude"}
        ],
        "geometryType": "esriGeometryPoint",
        "spatialReference": {"wkid": 4326},
        "name": "AireSensor_Layer",
        "description": "Datos de calidad del aire"
    }
}

def crear_actualizar_servicio(service_name, schema):
    """Crea o actualiza un servicio de entidades en ArcGIS Online"""
    try:
        # Buscar servicio existente
        search_result = gis.content.search(f"title:{service_name}", item_type="Feature Service")
        item = search_result[0] if search_result else None
        
        if item:
            logger.info(f"Servicio existente encontrado: {service_name}")
            flc = FeatureLayerCollection.fromitem(item)
            
            # Verificar si el servicio tiene capas
            if len(flc.layers) == 0:
                logger.warning("Servicio no tiene capas. Añadiendo capa...")
                update_result = flc.manager.add_to_definition({"layers": [schema]})
                logger.info(f"Capa añadida: {update_result}")
            
            return flc
        else:
            logger.info(f"Creando nuevo servicio: {service_name}")
            # Crear servicio vacío
            item = gis.content.create_service(
                name=service_name,
                service_description=schema["description"],
                has_static_data=False,
                max_record_count=2000,
                supported_query_formats="JSON",
                capabilities="Query,Editing,Create,Update,Delete",
                tags=["sensors", "iot", "puerto-salgar"]
            )
            flc = FeatureLayerCollection.fromitem(item)
            
            # Añadir definición de capa
            update_result = flc.manager.add_to_definition({"layers": [schema]})
            logger.info(f"Definición de capa añadida: {update_result}")
            
            return flc
    except Exception as e:
        logger.error(f"Error en crear_actualizar_servicio: {str(e)}")
        raise

def publicar_datos(flc, gdf):
    """Publica datos en un servicio de entidades"""
    try:
        # Verificar que el servicio tiene capas
        if len(flc.layers) == 0:
            logger.error("El servicio no tiene capas disponibles")
            return False
            
        layer = flc.layers[0]
        
        # Convertir GeoDataFrame a lista de features
        features = []
        for _, row in gdf.iterrows():
            # Manejar geometría vacía
            if row.geometry is None or row.geometry.is_empty:
                logger.warning("Geometría vacía encontrada. Saltando registro.")
                continue
                
            feature = {
                "geometry": {
                    "x": row.geometry.x,
                    "y": row.geometry.y,
                    "spatialReference": {"wkid": 4326}
                },
                "attributes": {
                    col: row[col] for col in gdf.columns if col != 'geometry'
                }
            }
            features.append(feature)
        
        # Insertar datos en lotes pequeños (mejor para estabilidad)
        batch_size = 50
        total_inserted = 0
        for i in range(0, len(features), batch_size):
            batch = features[i:i+batch_size]
            try:
                result = layer.edit_features(adds=batch)
                if 'addResults' in result:
                    success_count = sum(1 for res in result['addResults'] if res.get('success', False))
                    total_inserted += success_count
                    logger.info(f"Insertados {success_count}/{len(batch)} registros")
                else:
                    logger.error(f"Error en inserción: {result}")
            except Exception as e:
                logger.error(f"Error insertando lote: {str(e)}")
        
        logger.info(f"Total insertados: {total_inserted}/{len(features)} registros")
        return total_inserted > 0
    except Exception as e:
        logger.error(f"Error en publicar_datos: {str(e)}")
        return False

def preparar_dataframe(gdf):
    """Prepara el GeoDataFrame para publicación"""
    # Copiar para no modificar el original
    df = gdf.copy()
    
    # Eliminar solo la columna 'location' si existe
    if 'location' in df.columns:
        df = df.drop(columns=['location'])
    
    # Manejar campos de fecha - convertir a milisegundos desde época
    date_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    for col in date_cols:
        df[col] = (df[col].astype(np.int64) // 10**6).astype(int)
    
    return df

def crear_vista(flc, view_name):
    """Crea una vista para un servicio de entidades"""
    try:
        # Buscar si ya existe la vista
        search_result = gis.content.search(f"title:{view_name}", item_type="Feature Service")
        existing_view = search_result[0] if search_result else None
        
        if existing_view:
            logger.info(f"Vista existente encontrada: {view_name}")
            return FeatureLayerCollection.fromitem(existing_view)
        
        # Crear nueva vista
        logger.info(f"Creando vista: {view_name}")
        view_item = flc.manager.create_view(name=view_name)
        view_flc = FeatureLayerCollection.fromitem(view_item)
        
        logger.info(f"Vista creada exitosamente: {view_item.id}")
        return view_flc
    except Exception as e:
        logger.error(f"Error creando vista: {str(e)}")
        return None

def verificar_datos(layer, expected_count):
    """Verifica que los datos se hayan publicado correctamente"""
    try:
        # Esperar 10 segundos para que los datos se procesen
        logger.info("Esperando 10 segundos para verificación de datos...")
        time.sleep(10)
        
        # Consultar conteo de registros
        count_result = layer.query(where="1=1", return_count_only=True)
        
        if count_result == expected_count:
            logger.info(f"Verificación exitosa: {count_result} registros coinciden con lo esperado")
            return True
        else:
            logger.warning(f"Datos incompletos: Esperados {expected_count}, Encontrados {count_result}")
            return False
    except Exception as e:
        logger.error(f"Error verificando datos: {str(e)}")
        return False

def publicar_dataframes():
    # Diccionario de dataframes
    dataframes = {
        'tierra': df_tierra,
        'personas': df_personas,
        'tiempo': df_tiempo,
        'ruido': df_ruido,
        'aire': df_aire
    }
    
    # Publicar cada conjunto de datos
    for name, gdf in dataframes.items():
        service_name = SERVICE_NAMES[name]
        view_name = service_name + VIEW_SUFFIX  # Usando VIEW_SUFFIX definida
        schema = SCHEMAS[name]
        
        logger.info(f"Procesando: {service_name}")
        
        try:
            # Saltar dataframes vacíos
            if gdf.empty:
                logger.warning(f"Dataframe {service_name} vacío. Saltando publicación.")
                continue
                
            # Preparar dataframe
            prepared_df = preparar_dataframe(gdf)
            expected_count = len(prepared_df)
            
            # Crear/actualizar servicio
            flc = crear_actualizar_servicio(service_name, schema)
            
            # Publicar datos
            success = publicar_datos(flc, prepared_df)
            
            if success:
                logger.info(f"Publicación exitosa: {service_name}")
                
                # Crear vista
                view_flc = crear_vista(flc, view_name)
                if view_flc:
                    logger.info(f"Vista creada para {service_name}")
                else:
                    logger.warning(f"No se pudo crear vista para {service_name}")
                
                # Verificar datos en el servicio principal
                layer = flc.layers[0]
                data_ok = verificar_datos(layer, expected_count)
                
                # Verificar datos en la vista si se creó
                if view_flc:
                    view_layer = view_flc.layers[0]
                    view_data_ok = verificar_datos(view_layer, expected_count)
                
                logger.info(f"Proceso completo para {service_name}")
            else:
                logger.warning(f"Publicación parcial: {service_name}")
        except Exception as e:
            logger.error(f"Error publicando {service_name}: {str(e)}")

# Ejecutar la publicación
publicar_dataframes()

INFO:arcgis.gis._impl._portalpy:Retrieving roles(start=1, num=100)
INFO:__main__:Autenticado como: aplicaciones_puertosalgar
INFO:__main__:Procesando: TierraSensorData
INFO:__main__:Servicio existente encontrado: TierraSensorData
INFO:__main__:Insertados 1/1 registros
INFO:__main__:Total insertados: 1/1 registros
INFO:__main__:Publicación exitosa: TierraSensorData
INFO:__main__:Vista existente encontrada: TierraSensorData_View
INFO:__main__:Vista creada para TierraSensorData
INFO:__main__:Esperando 10 segundos para verificación de datos...
INFO:__main__:Esperando 10 segundos para verificación de datos...
INFO:__main__:Proceso completo para TierraSensorData
INFO:__main__:Procesando: PersonasSensorData
INFO:__main__:Servicio existente encontrado: PersonasSensorData
INFO:__main__:Insertados 1/1 registros
INFO:__main__:Total insertados: 1/1 registros
INFO:__main__:Publicación exitosa: PersonasSensorData
INFO:__main__:Vista existente encontrada: PersonasSensorData_View
INFO:__main__:Vista c

In [8]:
# Interpolación diaria - Versión simplificada
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features.analysis import interpolate_points
from datetime import datetime
import time

# ---------------------------
# CONFIGURACIÓN
# ---------------------------
# Conectar a la cuenta de ArcGIS Online
gis = GIS("https://www.arcgis.com", "aplicaciones_puertosalgar", "aplicaciones$2025*01")

# URLs de capas de entrada
AireSensorData_View = "https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/AireSensorData_View/FeatureServer/0"
TiempoSensorData_View = "https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/TiempoSensorData_View/FeatureServer/0"
TierraSensorData_View = "https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/TierraSensorData_View/FeatureServer/0"
aoiLayer_url = "https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/Mapa_PuertoSalgar2/FeatureServer/8"

# Crear objetos FeatureLayer
aire_layer = FeatureLayer(AireSensorData_View, gis)
tiempo_layer = FeatureLayer(TiempoSensorData_View, gis)
tierra_layer = FeatureLayer(TierraSensorData_View, gis)
aoi_layer = FeatureLayer(aoiLayer_url, gis)

# Nombres constantes para las capas (NUNCA CAMBIAR ESTOS NOMBRES)
NOMBRES_CAPAS_SALIDA = {
    "aire": "Interpolacion_Aire_Dashboard",
    "tiempo": "Interpolacion_Tiempo_Dashboard", 
    "tierra": "Interpolacion_Tierra_Dashboard"
}

# ---------------------------
# FUNCIONES PRINCIPALES
# ---------------------------
def buscar_capa_existente(nombre_capa):
    """Busca una capa existente por nombre exacto"""
    try:
        items = gis.content.search(f'title:"{nombre_capa}" AND owner:{gis.users.me.username}', 
                                 item_type="Feature Service", max_items=5)
        for item in items:
            if item.title == nombre_capa:
                return item
        return None
    except Exception as e:
        print(f"⚠️ Error buscando capa {nombre_capa}: {e}")
        return None

def eliminar_capa_existente(nombre_capa):
    """Elimina completamente una capa existente"""
    try:
        item = buscar_capa_existente(nombre_capa)
        if item:
            print(f"🗑️ Eliminando capa existente: {nombre_capa}")
            item.delete()
            time.sleep(3)  # Esperar para que se complete la eliminación
            return True
        return False
    except Exception as e:
        print(f"❌ Error eliminando capa {nombre_capa}: {e}")
        return False

def crear_interpolacion(cfg):
    """Crea interpolación directamente con el nombre final"""
    try:
        print(f"🔄 Creando interpolación: {cfg['name']}")
        
        # Verificar cantidad mínima de datos
        feature_count = cfg["layer"].query(return_count_only=True)
        print(f"📊 Registros disponibles: {feature_count}")
        
        if feature_count < 10:
            print(f"⚠️ Datos insuficientes: {feature_count} registros (mínimo 10 requeridos)")
            return None
        
        # Crear interpolación
        resultado = interpolate_points(
            input_layer=cfg["layer"],
            output_name=cfg["name"],
            field=cfg["field"],
            interpolate_option=5,  # IDW
            classification_type="GeometricInterval",
            num_classes=10,
            bounding_polygon_layer=aoi_layer,
            gis=gis
        )
        
        print(f"✅ Interpolación {cfg['name']} creada exitosamente")
        print(f"🔗 URL: {resultado.url}")
        
        return resultado
        
    except Exception as e:
        print(f"❌ Error creando interpolación {cfg['name']}: {e}")
        return None

def procesar_capa(cfg):
    """Procesa una capa de interpolación completa"""
    try:
        print(f"\n{'='*60}")
        print(f"🔄 Procesando {cfg['name']}")
        print(f"{'='*60}")
        
        # 1. Verificar datos de entrada
        feature_count = cfg["layer"].query(return_count_only=True)
        print(f"📊 Capa origen: {feature_count} registros")
        
        if feature_count < 10:
            print(f"❌ Datos insuficientes para {cfg['name']}: {feature_count} registros")
            return False
        
        # 2. Eliminar capa existente si existe
        capa_existe = buscar_capa_existente(cfg["name"])
        if capa_existe:
            print(f"📍 Capa existente encontrada: {cfg['name']}")
            print(f"🔗 URL actual: {capa_existe.url}")
            eliminar_capa_existente(cfg["name"])
            time.sleep(2)  # Esperar para asegurar que se eliminó completamente
        else:
            print(f"🆕 Primera ejecución - creando nueva capa")
        
        # 3. Crear nueva interpolación
        resultado = crear_interpolacion(cfg)
        
        if resultado:
            print(f"✅ {cfg['name']} procesada exitosamente")
            print(f"📊 ID del servicio: {resultado.id}")
            return True
        else:
            print(f"❌ Falló el procesamiento de {cfg['name']}")
            return False
            
    except Exception as e:
        print(f"❌ Error general procesando {cfg['name']}: {e}")
        import traceback
        print(f"Detalles: {traceback.format_exc()}")
        return False

def main():
    """Función principal"""
    print("🚀 INICIANDO ACTUALIZACIÓN DIARIA DE INTERPOLACIONES")
    print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"👤 Usuario: {gis.users.me.username}")
    
    # Configuración de capas
    layers_config = [
        {
            "name": NOMBRES_CAPAS_SALIDA["aire"], 
            "layer": aire_layer, 
            "field": "pm10"
        },
        {
            "name": NOMBRES_CAPAS_SALIDA["tiempo"], 
            "layer": tiempo_layer, 
            "field": "temperature"
        },
        {
            "name": NOMBRES_CAPAS_SALIDA["tierra"], 
            "layer": tierra_layer, 
            "field": "r_6_rwc"
        },
    ]
    
    # Procesar cada capa
    resultados = {}
    for cfg in layers_config:
        resultado = procesar_capa(cfg)
        resultados[cfg["name"]] = resultado
        
        # Pausa entre capas para evitar problemas de concurrencia
        if cfg != layers_config[-1]:  # No pausar después de la última
            time.sleep(5)
    
    # Resumen final
    print(f"\n{'='*70}")
    print("📋 RESUMEN FINAL")
    print(f"{'='*70}")
    
    for nombre, exito in resultados.items():
        estado = "✅ EXITOSA" if exito else "❌ FALLÓ"
        print(f"{estado:<12} {nombre}")
        
        # Mostrar URL actualizada
        if exito:
            item = buscar_capa_existente(nombre)
            if item:
                print(f"             🔗 {item.url}")
    
    exitosos = sum(1 for exito in resultados.values() if exito)
    total = len(resultados)
    
    print(f"\n🎯 Resultado final: {exitosos}/{total} capas actualizadas")
    
    if exitosos == total:
        print("🎉 ¡Todas las interpolaciones completadas exitosamente!")
    elif exitosos > 0:
        print("⚠️ Algunas interpolaciones fallaron - revisar logs arriba")
    else:
        print("❌ Todas las interpolaciones fallaron - revisar configuración")
    
    print(f"⏱️ Completado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    return exitosos, total

# ---------------------------
# EJECUCIÓN
# ---------------------------
if __name__ == "__main__":
    exitosos, total = main()

INFO:arcgis.gis._impl._portalpy:Retrieving roles(start=1, num=100)


🚀 INICIANDO ACTUALIZACIÓN DIARIA DE INTERPOLACIONES
📅 Fecha: 2025-09-23 03:11:50
👤 Usuario: aplicaciones_puertosalgar

🔄 Procesando Interpolacion_Aire_Dashboard
📊 Capa origen: 60 registros
🆕 Primera ejecución - creando nueva capa
🔄 Creando interpolación: Interpolacion_Aire_Dashboard
📊 Registros disponibles: 60


INFO:arcgis.gis._impl._portalpy:Retrieving roles(start=1, num=100)


✅ Interpolación Interpolacion_Aire_Dashboard creada exitosamente
🔗 URL: https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/Interpolacion_Aire_Dashboard/FeatureServer
✅ Interpolacion_Aire_Dashboard procesada exitosamente
📊 ID del servicio: 5e39af9ff92b4772a7fcd825b2664beb

🔄 Procesando Interpolacion_Tiempo_Dashboard
📊 Capa origen: 60 registros
🆕 Primera ejecución - creando nueva capa
🔄 Creando interpolación: Interpolacion_Tiempo_Dashboard
📊 Registros disponibles: 60


INFO:arcgis.gis._impl._portalpy:Retrieving roles(start=1, num=100)


✅ Interpolación Interpolacion_Tiempo_Dashboard creada exitosamente
🔗 URL: https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/Interpolacion_Tiempo_Dashboard/FeatureServer
✅ Interpolacion_Tiempo_Dashboard procesada exitosamente
📊 ID del servicio: 87a45ecb90a6464c8788fce0d4220d75

🔄 Procesando Interpolacion_Tierra_Dashboard
📊 Capa origen: 12 registros
🆕 Primera ejecución - creando nueva capa
🔄 Creando interpolación: Interpolacion_Tierra_Dashboard
📊 Registros disponibles: 12


ERROR:arcgis.geoprocessing._support:{"messageCode": "AO_40039", "message": "Not enough data to compute method. At least 10 points are required."}
ERROR:arcgis.geoprocessing._support:{"messageCode": "AO_100104", "message": "InterpolatePoints failed."}
ERROR:arcgis.geoprocessing._support:Failed to execute (InterpolatePoints).
ERROR:arcgis.geoprocessing._support:Failed.


❌ Error creando interpolación Interpolacion_Tierra_Dashboard: Job failed.
❌ Falló el procesamiento de Interpolacion_Tierra_Dashboard

📋 RESUMEN FINAL
✅ EXITOSA    Interpolacion_Aire_Dashboard
             🔗 https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/Interpolacion_Aire_Dashboard/FeatureServer
✅ EXITOSA    Interpolacion_Tiempo_Dashboard
             🔗 https://services3.arcgis.com/Rk47FK49lu9tP7as/arcgis/rest/services/Interpolacion_Tiempo_Dashboard/FeatureServer
❌ FALLÓ      Interpolacion_Tierra_Dashboard

🎯 Resultado final: 2/3 capas actualizadas
⚠️ Algunas interpolaciones fallaron - revisar logs arriba
⏱️ Completado: 2025-09-23 03:13:02
